In [13]:
import sys
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import pytz
import pandas as pd
import json 
import re


In [14]:
url='https://langclub.live/sessions'


In [15]:
def test_print(url):
    print(url)

In [16]:
def get_page(url):  
    HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Cache-Control": "max-age=0",
        }
    html = requests.get(url, headers=HEADERS)
    bs = BeautifulSoup(html.text, 'html.parser')
    return  bs

In [21]:


#filter data
def filter_langclub(bs):  
    script_json=bs.find('script', id='__NEXT_DATA__')
    json_object = json.loads(script_json.contents[0])
    list_seesion_big=json_object['props']['pageProps']['sessions']
    list_seesion=[]
    for session in list_seesion_big:
        dict_small=dict([ (i,session[i]) for i in session if i in "meeting_time"])#set(("title", "meeting_time")) ])
        list_seesion.append(dict_small)

    df_import = pd.DataFrame.from_records(list_seesion)
    df_import    



    #get current dateTime
    current_dateTime = datetime.now()
    now_timestamp = datetime.timestamp(current_dateTime)
    now_timestamp=int(now_timestamp)
    ling_timestamp=now_timestamp*1000
    ling_timestamp



    #filter by upcoming events only
    df_upcoming= df_import[df_import['meeting_time'] >= ling_timestamp].copy()

    #change data fromat
    df_upcoming["start_event"] = df_upcoming["meeting_time"].apply(lambda time_x: 
        datetime.fromtimestamp(time_x/1000, pytz.timezone("Europe/Kyiv")).isoformat())
    df_upcoming["end_event"] = df_upcoming["meeting_time"].apply(lambda time_x: 
        datetime.fromtimestamp(((time_x/1000)+3600), pytz.timezone("Europe/Kyiv")).isoformat())


    #final table
    df_upcoming['event_name']='Langclub'
    df_upcoming['description']=url
    df_upcoming['location']=None
    df_upcoming=df_upcoming[['event_name', 'description',  'start_event', 'end_event', 'location']]
    df_upcoming
    return df_upcoming



In [18]:
#be careful, we don't want to crash the site
#bs_langclub=get_page(url)

In [19]:
#df_upcoming_langclub=filter_langclub(bs_langclub)

In [20]:
#df_upcoming_langclub

,event_name,description,start_event,end_event,location
0,Langclub,https://langclub.live/sessions,2023-09-28T16:00:00+03:00,2023-09-28T17:00:00+03:00,https://langclub.live/sessions
1,Langclub,https://langclub.live/sessions,2023-09-29T16:00:00+03:00,2023-09-29T17:00:00+03:00,https://langclub.live/sessions
2,Langclub,https://langclub.live/sessions,2023-09-29T18:00:00+03:00,2023-09-29T19:00:00+03:00,https://langclub.live/sessions
3,Langclub,https://langclub.live/sessions,2023-09-29T21:00:00+03:00,2023-09-29T22:00:00+03:00,https://langclub.live/sessions
4,Langclub,https://langclub.live/sessions,2023-09-30T03:00:00+03:00,2023-09-30T04:00:00+03:00,https://langclub.live/sessions
5,Langclub,https://langclub.live/sessions,2023-09-30T16:00:00+03:00,2023-09-30T17:00:00+03:00,https://langclub.live/sessions
6,Langclub,https://langclub.live/sessions,2023-10-01T05:00:00+03:00,2023-10-01T06:00:00+03:00,https://langclub.live/sessions
7,Langclub,https://langclub.live/sessions,2023-10-01T16:00:00+03:00,2023-10-01T17:00:00+03:00,https://langclub.live/sessions
